## Regression tester

A version in jupyter notebook form to save time loading large csv files.

Regression test to compare two versions of outputs
Reads two csv files, old and new
Selects the columns of interest
Joins old and new on key columns, outer
Checks which records are in old only (left), new only (right) or both
Compares if the old and new values are the same within tolerance
Saves the ouotputs

In [12]:
import os

import pandas as pd

In [30]:
# Imputation file location and name
common_dir = "R:\\BERD Results System Development 2023\\DAP_emulation\\"

input_dir_new = "outputs\\output_long_form"
input_dir_new = "outputs\\output_gb_sas"

pref_old = "Frozen_Base_Data"
suff_old = "_2024-02-05_v471.csv"

pref_new = "output_gb_sas"
suff_new = "_2024-02-05_v471.csv"

# Output folder for all schemas
out_dir = "D:\\coding_projectsrandd_test_data\\"

# Columns to select
key_cols_old = ["reference", "200", "201"]
key_cols_new = ["reference", "instance", "200", "201"]
value_col = "211"
other_cols = ["status", "imp_marker",  "602", "referencepostcode", "postcodes_harmonised"]
tolerance = 0.001

In [32]:
read_in_dir = common_dir
read_in_path_old = os.path.join(common_dir, input_dir_old, pref_old + suff_old)
df_old = pd.read_csv(read_in_path_old)


# Remove "index" column if it exists
if "index" in df_old.columns:
    df_old = df.drop("index", axis=1)

df_old.head()

C:\Users\griffa1\Anaconda3\envs\resdev362\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (53) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,period_year,reference,formtype,form_status,instance,period,employment,wowenterprisereference,rusic,foreign_owner,...,headcount_res_m,headcount_res_f,headcount_tec_m,headcount_tec_f,headcount_oth_m,headcount_oth_f,emp_total,headcount_tot_m,headcount_tot_f,headcount_total
0,2022,49900000576,1,200,1,202212,7152,424697154,45320,GB,...,0.00,0.0,0.0,0.00,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,2022,49900000960,1,600,1,202212,2082,219265113,46342,DK,...,0.00,0.0,0.0,0.00,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2,2022,49900001029,1,200,1,202212,401,458968757,22110,US,...,1.75,0.0,0.0,0.00,1.0833,0.0833,2.9167,2.8333,0.0833,2.9167
3,2022,49900001032,1,200,1,202212,319,216215327,22190,GB,...,1.75,0.0,0.0,0.00,1.0833,0.0833,2.9167,2.8333,0.0833,2.9167
4,2022,49900001571,1,200,1,202212,260,216563078,13922,US,...,0.00,0.5,0.5,1.25,0.0000,0.2500,2.5000,0.5000,2.0000,2.5000


In [19]:
# Read the top 100 rows, inferrring the schema from csv
read_in_dir = common_dir
read_in_path_new = os.path.join(common_dir, input_dir_new, pref_new + suff_new)
df_new = pd.read_csv(read_in_path_new)


# Remove "index" column if it exists
if "index" in df_new.columns:
    df_new = df.drop("index", axis=1)

df_new.head()

,ref,period,entref,wow,selgrpno,sic07,form,emp,turn,f_owner,...,ctrl_gv,priv,ovss_oth,own_f,oth_sc,f_tot,ex_uk,ex_uk_gv,ex_uk_b,ex_total
0,49900000404,202212,9900000404,504666389,40,45310,6,242,195072,JP,...,1643.1825,1977.0701,620233.7278,4.000071e+06,4438.5223,4.628364e+06,8.059066e+06,57646.2254,1.166924e+06,9.283636e+06
1,49900000404,202212,9900000404,504666389,40,45310,6,242,195072,JP,...,0.0000,0.0000,0.0000,0.000000e+00,0.0000,0.000000e+00,0.000000e+00,0.0000,0.000000e+00,0.000000e+00
2,49900000576,202212,9900000576,424697154,40,45320,1,7152,924764,GB,...,0.0000,0.0000,0.0000,0.000000e+00,0.0000,0.000000e+00,0.000000e+00,0.0000,0.000000e+00,0.000000e+00
3,49900001029,202212,9900001029,458968757,14,22110,1,401,197907,US,...,0.0000,0.0000,0.0000,2.445432e+05,0.0000,2.445432e+05,0.000000e+00,0.0000,0.000000e+00,0.000000e+00
4,49900001032,202212,9900001032,216215327,14,22190,1,319,37475,GB,...,0.0000,0.0000,0.0000,2.445432e+05,0.0000,2.445432e+05,0.000000e+00,0.0000,0.000000e+00,0.000000e+00


In [26]:
# filter conditions
def get_mask(df:pd.DataFrame, col:str, values:list):
    return df[col].isin(values)

def filter_df(df:pd.DataFrame, col:str, values:list):
    return df.copy().loc[df[col].isin(values)]

# filter_df(df_old, "formtype", [1]).head()

# # Filter good statuses only
# imp_markers_to_keep = ["TMI", "CF", "MoR", "constructed"]
# df_old = filter_df(df_old, "imp_marker", imp_markers_to_keep)
# df_new = filter_df(df_new, "imp_marker", imp_markers_to_keep)

In [31]:

# sizes
print(f"Old size: {df_old.shape}")
print(f"New size: {df_new.shape}")

#%% Join
df_merge = df_old.merge(
    df_new,
    on=key_cols,
    how="outer",
    suffixes=("_old", "_new"),
    indicator=True
)
#%% Compare the values
df_merge["value_different"] = (
    (df_merge[value_col + "_old"] - df_merge[value_col + "_new"])**2 > tolerance**2
)

# %% Save output
write_csv(out_fol + out_file, df_merge)

Old size: (8748, 68)
New size: (13959, 38)


KeyError: 'reference'

## Calculate the schema

In [ ]:
# Initially, the schema is empty
base_str = ""
add_in_str = f'[{col}]\nold_name = "{col}"\nDeduced_Data_Type = "{schema[col]}"\n\n'


# Iterate through columns, adding to the string which will be written to toml
for col in schema:
    base_str += add_in_str

In [ ]:
# Output the schema toml file
mypath = os.path.join(out_dir, pref + "_schema.toml")
text_file = open(mypath, "w")
text_file.write(S)
text_file.close()